In [6]:
#importing OpenCV (cv2) and pytesseract (Install tesseract-OCR before it)
import cv2
import pytesseract

In [7]:
list1 = []
grid1 = []
line = []

In [8]:
img = cv2.imread('inputfile/data_table_example2.png')
im = img.copy()

imgray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
ret1,thresh1 = cv2.threshold(imgray,120,255,cv2.THRESH_BINARY)
ret2,thresh2 = cv2.threshold(imgray,120,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)

In [9]:
# imgray

In [10]:
cv2.imshow('Input Image',img)
cv2.imshow('Gray Image',imgray)

# cv2.imshow('Canny Image',edge)

cv2.imshow('Output Image1',thresh1)
cv2.imshow('Output Image2',thresh2)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
image1, contours1, hierarchy1 = cv2.findContours(thresh2,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

In [12]:
# contours1[0]
# cv2.arcLength(contours1[0],True)

In [13]:
for i in range(len(contours1)):
    perimeter=cv2.arcLength(contours1[i],True)
    if perimeter > 250 and perimeter < 2000:    #th1 = 100
#         cv2.drawContours(im, contours1, i, (0,255,0), 2)
        cv2.polylines(im,[contours1[i]],True,(0,255,0))
#         print(contours1[i])
        list1.append(i)

In [14]:
cv2.imshow('Input Image',img)
cv2.imshow('Output Image',im)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [15]:
inv_list1 = list1[::-1]

In [16]:
len(inv_list1)

37

In [17]:
# cv2.boundingRect(contours1[156])

In [18]:
# img.shape[0]

In [19]:
# for x in list1[::-1]:
for i in range(len(list1)):
    this = inv_list1[i]
    this_tp_bdr = cv2.boundingRect(contours1[this])[1]
    if i < len(inv_list1)-1:
        next = inv_list1[i+1]
        next_tp_bdr = cv2.boundingRect(contours1[next])[1]
    else:
        next_tp_bdr = img.shape[0]
    
#     print(contours1[x])
#     cv2.polylines(im,[contours1[x]],True,(0,255,0))
    x,y,w,h = cv2.boundingRect(contours1[inv_list1[i]])
    cropped = imgray[y:y+h,x:x+w]
#     print(x,y,w,h)

    res = cv2.resize(cropped,None,fx=1.5, fy=1.5, interpolation= cv2.INTER_LINEAR)
    text = pytesseract.image_to_string(res, lang = 'eng')
    text = text.replace('\n',' ')
#     print(text)
#     output = io.StringIO()
#     output.write(text)
#     row.append(output.getvalue())
    line.append(text)

    if((this_tp_bdr + 4) < next_tp_bdr):
        grid1.append(line)
        line = []

In [20]:
print(grid1)

[['Transmittance (%T)'], ['Trial #1', 'Trial #2', 'Trial #3', 'Trial #4', 'Trial #5'], ['77.23', '74.50', '64.88', '75.27', '54.66'], ['85.23', '92.82', '78.91', '60.71', '57.96'], ['88.39', '100.05', '73.66', '66.51', '64.54'], ['80.71', '100.05', '68.29', '64.91', '52.96'], ['82.66', '117.18', '71.01', '56.91', '46.95'], ['72.55', '115.40', '65.72', '66.03', '55.38']]


In [21]:
# grid1 = []

In [22]:
cv2.imshow('Original Image',img)
cv2.imshow('Gray Image',imgray)
cv2.imshow('Output Image', im)
cv2.imshow('Image_1', image1)

# cv2.imshow('Cropped Image',cropped)
# cv2.imshow('Resized Image',res)

cv2.waitKey(0)
cv2.destroyAllWindows()

#### Here I'm converting data-array to CSV file

In [25]:
# import csv
# with open("data.csv", "a", newline='') as csvfile:
#     writer = csv.writer(csvfile)
#     writer.writerows(grid1)

#### Here I'm converting data-array to .xlsx file

In [26]:
# import xlsxwriter module 
import xlsxwriter 

workbook = xlsxwriter.Workbook('Output.xlsx') 

worksheet = workbook.add_worksheet("My sheet") 

# Start from the first cell.
# Rows are zero indexed. 
row = 0

# Iterate over the data and write it out row by row. 
for rows in (grid1): 
    col = 0
    for item in rows: 
        worksheet.write(row, col, item) 
        col += 1
    row += 1